# Web Information Retrieval
## Introduction to search engines

### DAY 1: Student version
### Introduction & First Steps

The goal of this first day is to go through the data and propose some ideas for the creation of your first search engine.

The notebook is divided into several steps:
- Query on the data
- Data extraction
- Data exploration
- Data Visualization
- Ideas suggestion for the search engine

The data are from the stack exchange forum. They are available here : https://archive.org/details/stackexchange


In order to gain time, you can already download 7 zip file of datascience forum (datascience.stackexchange.com.7z). If you work on Colab, we suggest you to push it on a specific directory (it may take few minutes to load).





# Query on the data

Perform SQL queries to explore the data on the surface.

The queries are to be done directly from this link: https://data.stackexchange.com/datascience/query/new

For each query, you must test it on the site and then rewrite it in the associated cell. A capture of the results and a comment is requested in the report.

You can find a cheat sheet on SQL basic syntax at: https://res.cloudinary.com/dyd911kmh/image/upload/v1675360372/Marketing/Blog/SQL_Basics_For_Data_Science.pdf


In [ ]:
# How many lines are in the Posts, Tags, Comments tables ?

In [ ]:
# How many comments have there been since the beginning of the year ?

In [ ]:
# How many users are there ?

In [ ]:
# How many new users are there each year since 2020 ?

In [ ]:
# What is the Content of the smallest Post ?

In [ ]:
# What is the most voted post ?

In [ ]:
# What are the 10 most frequent tags in 2022? (ordered from most to least frequent)

## Initialize the environnement

In [ ]:
## Install the library for extracting 7 zip file
!pip install py7zr --quiet

In [ ]:
## Import the libraries

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
import py7zr
import os

There is a simple method for extracting the data directly from the website zip repo. It writes the different files directly in your google drive.

In [ ]:
# Only if you use Colab

drive.mount('/content/drive')

In [ ]:
# To adapt to your Drive/local directory

MAIN_PATH = '/content/drive/MyDrive/TP Centrale'
DATA_PATH = '/content/drive/MyDrive/TP Centrale/data'

In [ ]:
if not os.path.isdir(MAIN_PATH):
  os.mkdir(MAIN_PATH)
if not os.path.isdir(MAIN_PATH):
  os.mkdir(DATA_PATH)

In [ ]:
archive = py7zr.SevenZipFile(os.path.join(MAIN_PATH, 'datascience.stackexchange.com.7z'), mode='r')
archive.extractall(path=os.path.join(MAIN_PATH, 'data'))
archive.close()

## Data Extraction

The data is in XML format. Why do you think the developers preferred this format? What could have been other possible formats? (**answer expected in the report**)

Here is what a raw file looks like:


In [ ]:
with open(os.path.join(DATA_PATH, 'Tags.xml'), 'r') as f:
  raw_xml = f.read()
print(raw_xml)

### XML Extraction example for Tags file
Here is a method to read these XML files easily as a pandas DataFrame.

In [ ]:
tags = pd.read_xml(os.path.join(DATA_PATH, 'Tags.xml'), parser="etree", encoding="utf8")

In [ ]:
tags

There are several other files in the extracted folder:

In [ ]:
os.listdir(DATA_PATH)

In the next part you will do some exploration on them, starting with the more important one: Posts.xml. It contains the many posts of the selected topic: "data science".

## Data Exploration

In this part, you have to work on Posts.xml file.


You can find a cheat sheet of pandas basic syntax at: https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf

In [ ]:
# Open the Posts.xml file

posts = pd.read_xml(os.path.join(DATA_PATH, 'Posts.xml'), parser="etree", encoding="utf8")
posts

In [ ]:
# What is the size of this file ?

In [ ]:
# What is the schema of the posts table ?

In [ ]:
# Give the mean number of new posts by user 

In [ ]:
# Run some other queries to get a better understanding of the "Posts" table

## Data Visualization 

Exploring data only through queries has its limits. 

An important step in understanding the data is in the visualization. It is important to think about what information I want to put forward and how.

You have the matplotlib and seaborn libraries available:
- https://matplotlib.org/stable/index.html
- https://seaborn.pydata.org/


You can find some cheatsheets at:
 - for matplotlib: https://matplotlib.org/cheatsheets/
 - for seaborn: https://res.cloudinary.com/dyd911kmh/image/upload/v1676302629/Marketing/Blog/Seaborn_Cheat_Sheet.pdf


#### Example of matplotlib use

In [ ]:
tags_5 = tags.head(5)

fig, ax = plt.subplots(figsize=(8, 5), layout='constrained')

plt.bar(x=tags_5["TagName"], height=tags_5["Count"], width=0.4)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Posts count')
ax.set_title('Distribution of Posts count by Tag')
ax.set_ylim(0, 12_000)

plt.show()

#### Example of seaborn use

In [ ]:
tags_5 = tags.head(5)

fig, ax = plt.subplots(figsize=(8, 5), layout='constrained')

sns.barplot(x=tags_5["TagName"], y=tags_5["Count"], width=0.4)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Posts count')
ax.set_title('Distribution of Posts count by Tag')
ax.set_ylim(0, 12_000)

plt.show()

These were 2 examples of distribution visualization via histograms.

On Seaborn, several types of fairly intuitive graphs are available. They are classified by type of input data:
- Visualize 2 continuous variables: relplot
- Visualize 1 continuous variable in relation to a categorical variable: displot
- Visualize 2 categorical variables : catplot

![image.png](https://seaborn.pydata.org/_images/function_overview_8_0.png)


More examples can be found here : https://seaborn.pydata.org/tutorial/function_overview.html


In [ ]:
# Viz 1 : Plot the distribution of number of posts created by users

# Directly use histplot takes too long because of the number of users.
# Better to aggregate and then use barplot

# Doesn't take into account users that have never created posts

posts_count = posts.groupby("OwnerUserId", as_index=False)["Id"].count()
posts_count = posts_count.rename(columns={"Id": "postNumber"})
posts_count_number = posts_count.groupby("postNumber", as_index=False).count()

sns.barplot(x="postNumber", y="OwnerUserId", data=posts_count_number)
plt.xlim((0, 10))
plt.ylabel("Number of New Posts")
plt.title("Distribution of the number of users by new posts number")

In [ ]:
# Viz 2 : Plot the distribution of number of posts by months
posts["month_creation_date"] = pd.to_datetime(posts.CreationDate).dt.month

posts_by_month = posts.groupby("month_creation_date", as_index=False)["OwnerUserId"].count()

sns.barplot(x="month_creation_date", y="OwnerUserId", data=posts_by_month)
plt.ylabel("Number of New Posts")
plt.title("Distribution of the new posts number by months")

In [ ]:
# Viz 3 : Plot the distribution of number of characters by posts and by year starting from 2020
posts["CreationDateYear"] = pd.to_datetime(posts.CreationDate).dt.year

posts_sup2020 = posts[posts.CreationDateYear > 2020]

posts_sup2020["nbCaracters"] = posts_sup2020.Body.str.len()

sns.histplot(x="nbCaracters", hue="CreationDateYear", data=posts_sup2020)
plt.xlim((0, 5000))
plt.ylabel("Number of Characters")
plt.title("Distribution of the number of characters by posts and by users")

In [ ]:
# Add other visualization figures that can help you to decide the search engine architecture

## Exploring Other Files

The purpose of this part is to understand the other files using the same process and data collection site: https://archive.org/details/stackexchange

You will describe the purpose and content of each file to show your understanding of the subject. You will also make a relational diagram of the different tables directly in the report.

In [ ]:
# Open, explore and visualize other XML files to suggest features for your search engines

## Suggest ideas for the search engine

Using your work and knowledge of this database, propose a structure for your search engine in the report. You do not need to code it entirely yet, simply write down some ideas you may already have on how you could build a great search engine, which data to use and how to use it.

Reminder: The aim is to provide a search engine able to find information from any of the files with textual data in it (posts, comments, ...) on the datascience topic. Some files might be better than others when it comes to searching for information, some metadata could be used aswell... 